# 1. Import libraries

In [2]:
import pandas as pd
import numpy as np

import fastf1
from fastf1.ergast import Ergast

from tqdm import tqdm
import os

import sys
sys.path.append('../')

from src.race_prediction_model.extract import *
from src.race_prediction_model.feature_engineering import add_features_to_results

# Config less verbosity
fastf1.set_log_level('ERROR')

# 2. Data loading

Extraemos los circuitos (esto lo necesitamos antes)

In [ ]:
# Poner esto también en A1-EDA-Clustering

try:
    df_races = pd.read_csv('../data/output/races.csv')

except FileNotFoundError:
    print("Data not found: Extracting data...")
    df_races = extract_races_dataframe(2010, end=2024, save=True)

In [6]:
df_races

,season,round,circuitId
0,2010,1,bahrain
1,2010,2,albert_park
2,2010,3,sepang
3,2010,4,shanghai
4,2010,5,catalunya
...,...,...,...
300,2024,20,rodriguez
301,2024,21,interlagos
302,2024,22,vegas
303,2024,23,losail
